# Load all the libraries


**Import required libraries and modules**

In [ ]:
import warnings
warnings.filterwarnings('ignore') #, category=UserWarning)
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy





from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from datetime import datetime
import time

import os
import ast
from pathlib import Path

In [ ]:
start_time_notebook = time.time()

# Define the path of the data


**Import required libraries and modules**

- The directory is mounted from the **bscearth** server
- Experiment: EC-Earth **A5CW** 
- CMORFILES
- piControl; Pre-Industrial (pi)
- Spatial Resolution: **r1i1p1f1**
- Model: Ocean Only **Omon**
- Temporal: **Monthly**

In [ ]:
gpfs = '/gpfs/projects/bsc32/bsc032196'
list_dir = gpfs + '/esarchive/exp/ecearth/a5cw/original_files/cmorfiles/CMIP/EC-Earth-Consortium/EC-Earth3-CC/piControl/r1i1p1f1/Omon/'
print("MAIN DIRECTORY: ",list_dir)

pisces_files = os.listdir(list_dir)
print("")                   
print("List of subdirectories (Variables): ",pisces_files)



parent_dir = Path().resolve().parent
print("Parent Directory:", parent_dir)

epoch_range = '*gn_19*' #*gn_199* 1990s #*gn_199* 1990s 
file_ext = '.nc'
if os.path.exists(f'{parent_dir}/plots/Heatmap_{epoch_range}') == False:
    os.makedirs(f'{parent_dir}/plots/Heatmap_{epoch_range}', exist_ok=True) 
if os.path.exists(f'{parent_dir}/output_time/Heatmap_{epoch_range}') == False:
    os.makedirs(f'{parent_dir}/output_time/Heatmap_{epoch_range}', exist_ok=True)


In [ ]:
print(list_dir)
print(pisces_files)

# Extract the variables

- Z(t) = INTPP : net_primary_mole_productivity_of_biomass_expressed_as_carbon_by_phytoplankton [mol m-2 s-1]
- Z(t-1) = INTPP Lag 1: 
- A(t) = NO3 3D (Column integrate 200m): mole_concentration_of_nitrate_in_sea_water [mol m-3]
- B(t) = PO4 3D (Column integrate 200m): mole_concentration_of_dissolved_inorganic_phosphorus_in_sea_water [mol m-3]
- C(t) = Si 3D (*Column integrate 200m*)
- D(t) = THETAO 3D (Column integrate 200m): sea_water_potential_temperature [degC]
- E(t) = MLOTST: ocean_mixed_layer_thickness_defined_by_sigma_t [m]
- G(t) = DFE 3D (Column integrate 200m): mole_concentration_of_dissolved_iron_in_sea_water [mol m-3]

- H(t) = ZMESOOS (Surface) Only from 1959..onwards: Surface Mole Concentration of Mesoozooplankton expressed as Carbon in Sea Water [mol m-3]
- I(t) = ZMICROOS (Surface) Only from 1959..onwards: Microzooplankton [mol m-3]
- *J(t) = ZMISC ?*
- *K(t) = PHOT (Column integrate 200m)?*
- L(t) = RSNTDS (Surface): Net Surface Downward Shortwave Radiation at Surface [W m-2] 
- M(t) = UMO (at 50 m): ocean_mass_x_transport [kg s-1]
- O(t) = VMO (at 50 m): ocean_mass_y_transport [kg s-1]
- P(t) = WMO (at 50 m): upward_ocean_mass_transport [kg s-1]


Multivar autoregressive model with INTPP vs. 6 external variables 

Z(t) = z*Z(t-1) + a*A(t)....e*E(t)....l*L(t)

ARIMA:  with (p,d,q) Model

LSTM: Long Short-Term Memory

Prediction for PISCES variables
Z(t) = Z(t-1) + A....L

**Start the wall clock**

In [ ]:
start_time_load = time.time()

# Print the variables and their positions in the list
list_multivar = ['INTPP','NO3' ,'PO4','SI','THETAO','MLOTST','DFE','ZMESOOS','ZMICROOS',
                 'RSNTDS','VMO','WMO','UMO']
#list_multivar = ['INTPP','NO3', 'PO4'] #,'VMO','WMO','ZMESOOS','ZMICROOS','NO3','UMO','PO4','SI','DFE','MLOTST','RSNTDS','THETAO']
#list_multivar = ['INTPP','DFE','RSNTDS']


epoch = epoch_range + file_ext

for i in range(len(pisces_files)):
    
    if pisces_files[i].upper() in list_multivar:
        
        
        #os.environ["HDF5_DISABLE_VERSION_CHECK"] = "2"
        warnings.filterwarnings('ignore' , category=UserWarning)
        files = list_dir+pisces_files[i]+'/gn/v20221230/'
        #print("Variable Directory ", files)
        list_nc_files = os.listdir(files)
        #print("All Files for this VARIABLE",list_nc_files)
        if pisces_files[i].upper() == 'INTPP':
            print("Position, VARIABLE:  ",i,pisces_files[i].upper())
            intpp = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
            #intpp = xr.open_mfdataset(files+epoch).isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'NO3':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            no3 = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'PO4':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            po4 = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'SI':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            si = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'THETAO':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            theta0 = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'MLOTST':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            mlotst = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'DFE':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            dfe = xr.open_mfdataset(files+epoch,combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'ZMESOOS':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            zmesoos = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'ZMICROOS':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            zmicroos = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'RSNTDS':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            rsntds = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'WMO':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            wmo = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'UMO':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            umo = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))
        if pisces_files[i].upper() == 'VMO':
            print("VARIABLE DIR:  ",i,pisces_files[i].upper())
            vmo = xr.open_mfdataset(files+epoch, combine='by_coords')#.isel(time=slice(None, None, 12))


                    
end_time_load = time.time()
time_used_load = (end_time_load - start_time_load)
print("Time used for entire LOAD..../zzzz", f'{time_used_load:.4f}', "Seconds")
print("Time used for entire LOAD..../zzzz", f'{time_used_load/60:.4f}', "Minutes")
print("Time used for entire LOAD..../zzzz", f'{time_used_load/3600:.4f}', "Hours")
# Open a file in write mode
with open(f"{parent_dir}/output_time/Heatmap_{epoch_range}/output_time_heatmap_{epoch_range}_load.txt", "w") as file:
    # Redirect print output to the file
    print("Time used for LOAD..../zzzz", f'{time_used_load:.4f}', "Seconds", file=file)
    print("Time used for LOAD..../zzzz", f'{time_used_load/60:.4f}', "Minutes", file=file)
    print("Time used for LOAD..../zzzz", f'{time_used_load/3600:.4f}', "Hours", file=file)

# Time Series Extractor


In [ ]:
%%capture --no-stderr --no-display
warnings.filterwarnings('ignore' , category=UserWarning)

class TimeSeriesExtractor:
    def __init__(self, datasets, k_factors, train_start_year,
                 train_end_year, test_start_year, test_end_year, geo_locations):
        self.datasets = datasets  # A dictionary of datasets
        self.k_factors = k_factors  # Dictionary of scaling factors for each variable
        self.train_start_year = train_start_year
        self.train_end_year = train_end_year
        self.test_start_year = test_start_year
        self.test_end_year = test_end_year
        self.geo_locations = geo_locations

    def extract_time_series(self, var_name, i_index, j_index, lev=None, method='nearest'):
        """
        Extracts time series data from the specified dataset at the given indices.
        
        :param var_name: The variable name in the dataset.
        :param i_index: The i index of the target point.
        :param j_index: The j index of the target point.
        :param lev: Optional level index for 3D variables.
        :param method: The method to use for selection ('nearest' by default).
        :return: The extracted time series.
        """
        warnings.filterwarnings('ignore' , category=UserWarning)
        
        print("LEVEL??",lev)
        if lev is not None:
            
            lev_end = lev
            lev_start = 0
            print("From 0 to LEVEL??",lev_start,lev_end)
            time_series = self.datasets['data'][var_name].isel(i=i_index, j=j_index,lev=slice(lev_start, lev_end + 1)).mean(dim='lev')
        else:
            #print("self.datasets['data'][var_name].isel(i=i_index, j=j_index)",
            #      self.datasets['data'][var_name].isel(i=i_index, j=j_index))
            time_series = self.datasets['data'][var_name].isel(i=i_index, j=j_index)
        
        return time_series[var_name]

    def normalize_time_series(self, time_series):
        """
        Normalizes a given time series array to have zero mean and unit variance.
        
        :param time_series: The time series data array to normalize.
        :return: The normalized time series data array.
        """
        
        #print("time_series",time_series)
        #print("type(time_series)",type(time_series))
        mean = np.mean(time_series)
        std = np.std(time_series)
        normalized_series = (time_series - mean) / (std + 0.01)
        return normalized_series

    def k_scaled_timeseries(self, time_series, k_factor):
        """
        Transform with k_factor multiplication, addition with constant, 
        logarithmic transformation and by adding a constant value of 12
        
        :param time_series: The time series data array.
        :return: The k-scaled time series data array.
        """
        k_scaled_series = np.log1p(np.abs(time_series) * k_factor + np.exp(2)) + 12
        return k_scaled_series

    def replace_nan_with_zero(self, time_series):
        """
        Checks for NaN values in the time series and replaces them with 0.
        
        :param time_series: The time series data array.
        :return: The time series data array with NaNs replaced by 0.
        """
        time_series[np.isnan(time_series)] = 0
        return time_series

    def get_time_series_for_multiple_points(self, index_pairs):
        """
        Extracts time series data for multiple points specified by index pairs.
        
        :param index_pairs: A list of (i_index, j_index) pairs.
        :return: A dictionary with keys as (i_index, j_index) tuples and values as time series data dictionaries.
        """
        all_time_series_data = {}
        for (i_index, j_index) in index_pairs:
            print("Geo-location: ", i_index, j_index)
            print("self.geo_locations['geo_points']:", self.geo_locations['geo_points'].items())
            print("self.geo_locations['geo_points']:", self.geo_locations['geo_points'].keys())
            print("f'({i_index},{j_index})'",f'({i_index}, {j_index})')
            print("get.geo_locations['geo_points'](f'({i_index}, {j_index})')",self.geo_locations['geo_points'][(i_index, j_index)])
            time_series_data = {}
            for key, variable in self.datasets['variables'].items():
                lev = variable.get('lev', None)
                print("variable['name'] ",variable['name'])
                time_series = self.extract_time_series(variable['name'], i_index, j_index, lev).values
                
                # Replace NaN values with 0
                time_series = self.replace_nan_with_zero(time_series)
                
                # Normalize time series
                normalized_series = self.normalize_time_series(time_series)
                
                # Apply k factor scaling
                k_factor = self.k_factors.get(key, 1)
                k_scaled_series = self.k_scaled_timeseries(time_series, k_factor)
                
                # Replace NaN values with 0 again after scaling
                normalized_series = self.replace_nan_with_zero(normalized_series)
                k_scaled_series = self.replace_nan_with_zero(k_scaled_series)
                
                # Split the data into training and testing based on specified years
                train_mask = (self.datasets['data'][variable['name']].time.dt.year >= self.train_start_year) & (self.datasets['data'][variable['name']].time.dt.year <= self.train_end_year)
                test_mask = (self.datasets['data'][variable['name']].time.dt.year >= self.test_start_year) & (self.datasets['data'][variable['name']].time.dt.year <= self.test_end_year)
                
                time_series_data[key] = {
                    'normalized_train': normalized_series[train_mask],
                    'normalized_test': normalized_series[test_mask],
                    'k_scaled_train': k_scaled_series[train_mask],
                    'k_scaled_test': k_scaled_series[test_mask]
                }
            all_time_series_data[(i_index, j_index)] = time_series_data
        return all_time_series_data

    def time_function_execution(self, func, *args, **kwargs):
        """
        Measures the wall clock time taken by a function.
        
        :param func: The function to execute.
        :param args: Positional arguments to pass to the function.
        :param kwargs: Keyword arguments to pass to the function.
        :return: The result of the function and the time taken (in seconds).
        """
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result, elapsed_time

    def convert_to_dataframe(self, time_series_data):
        """
        Converts the time series data to pandas DataFrames.

        :param time_series_data: The dictionary containing the time series data.
        :return: Two pandas DataFrames for training and testing data.
        """
        df_train = pd.DataFrame()
        df_test = pd.DataFrame()

        # Use the time index from the dataset
        for point_key, series_dict in time_series_data.items():
            for variable_key, series_data in series_dict.items():
                train_col_names = {
                    'normalized': f"{variable_key}_normalized_train_{point_key}",
                    'k_scaled': f"{variable_key}_k_scaled_train_{point_key}"
                }
                test_col_names = {
                    'normalized': f"{variable_key}_normalized_test_{point_key}",
                    'k_scaled': f"{variable_key}_k_scaled_test_{point_key}"
                }
                
                df_train[train_col_names['normalized']] = series_data['normalized_train']
                df_test[test_col_names['normalized']] = series_data['normalized_test']
                df_train[train_col_names['k_scaled']] = series_data['k_scaled_train']
                df_test[test_col_names['k_scaled']] = series_data['k_scaled_test']

        # Assign the date range as the index of the DataFrame
        time_train_index = self.datasets['data'][variable_key].time.sel(time=(self.datasets['data'][variable_key].time.dt.year >= self.train_start_year) & (self.datasets['data'][variable_key].time.dt.year <= self.train_end_year))
        time_test_index = self.datasets['data'][variable_key].time.sel(time=(self.datasets['data'][variable_key].time.dt.year >= self.test_start_year) & (self.datasets['data'][variable_key].time.dt.year <= self.test_end_year))
        
        df_train.index = time_train_index
        df_test.index = time_test_index

        return df_train, df_test

    def add_lagged_series(self, df_train, df_test, variable, point):
        """
        Adds a lag-1 shifted series for the specified variable and point.

        :param df_train: The training DataFrame.
        :param df_test: The testing DataFrame.
        :param variable: The variable for which to add the lagged series.
        :param point: The point (i, j) for which to add the lagged series.
        """
        point_key = point
        variable_key = variable

        train_col = {
            'normalized': f"{variable_key}_normalized_train_{point_key}",
            'k_scaled': f"{variable_key}_k_scaled_train_{point_key}"
        }
        test_col = {
            'normalized': f"{variable_key}_normalized_test_{point_key}",
            'k_scaled': f"{variable_key}_k_scaled_test_{point_key}"
        }
        
        lagged_column_train = {
            'normalized': f"{variable_key}_lag1_normalized_train_{point_key}",
            'k_scaled': f"{variable_key}_lag1_k_scaled_train_{point_key}"
        }
        lagged_column_test = {
            'normalized': f"{variable_key}_lag1_normalized_test_{point_key}",
            'k_scaled': f"{variable_key}_lag1_k_scaled_test_{point_key}"
        }
        
        df_train[lagged_column_train['normalized']] = df_train[train_col['normalized']].shift(1)
        df_test[lagged_column_test['normalized']] = df_test[test_col['normalized']].shift(1)
        df_train[lagged_column_train['k_scaled']] = df_train[train_col['k_scaled']].shift(1)
        df_test[lagged_column_test['k_scaled']] = df_test[test_col['k_scaled']].shift(1)

        df_train[lagged_column_train['normalized']].iloc[0] = np.nan  # First value is NaN
        df_test[lagged_column_test['normalized']].iloc[0] = df_train[lagged_column_train['normalized']].iloc[-1]  # Initial test value is the last train value
        df_train[lagged_column_train['k_scaled']].iloc[0] = np.nan  # First value is NaN
        df_test[lagged_column_test['k_scaled']].iloc[0] = df_train[lagged_column_train['k_scaled']].iloc[-1]  # Initial test value is the last train value

    def apply_cross_correlation(self, series1, series2, max_lag=3):
        series1 = np.array(series1)
        series2 = np.array(series2)
        cross_corr = {}

        for lag in range(-max_lag, max_lag + 1):
            if lag < 0:
                lagged_series1 = series1[:lag]
                lagged_series2 = series2[-lag:]
            elif lag > 0:
                lagged_series1 = series1[lag:]
                lagged_series2 = series2[:-lag]
            else:
                lagged_series1 = series1
                lagged_series2 = series2

            corr = scipy.stats.pearsonr(lagged_series1,lagged_series2)
            #corr = np.correlate(lagged_series1 - np.mean(lagged_series1), lagged_series2 - np.mean(lagged_series2), mode='valid')
            #norm_factor = np.std(lagged_series1) * np.std(lagged_series2)
            cross_corr[lag] = corr.statistic
            #if norm_factor > 0:
            #    cross_corr[lag] = corr[0] / norm_factor
            #else:
            #    cross_corr[lag] = np.nan
        
        return cross_corr

    def calculate_cross_correlations(self, time_series_data, max_lag=3):
        results = []

        for point_key, series_dict in time_series_data.items():
            for var1_key, series1_data in series_dict.items():
                for var2_key, series2_data in series_dict.items():
                    if var1_key != var2_key:  # Avoid self-correlation
                        cross_corr = self.apply_cross_correlation(series1_data['normalized_train'], series2_data['normalized_train'], max_lag)
                        for lag, corr_value in cross_corr.items():
                            results.append({
                                'point': point_key,
                                'geopoint':self.geo_locations['geo_points'][point_key],
                                'var1': var1_key,
                                'var2': var2_key,
                                'lag': lag,
                                'correlation': corr_value
                            })

        return pd.DataFrame(results)
    

# Example usage:
datasets = {
    'data': {
        'intpp': intpp,
        'no3': no3,
        'po4': po4,
        'si': si,
        'thetao': theta0,
        'mlotst': mlotst,
        'dfe': dfe,
        'zmicroos': zmicroos,
        'zmesoos': zmesoos,
        'rsntds': rsntds,
        'umo': umo,
        'vmo': vmo,
        'wmo': wmo
        
    },
    
    'variables': {
        'intpp': {'name': 'intpp'},
        'no3': {'name': 'no3', 'lev': 6},
        'po4': {'name': 'po4', 'lev': 6},
        'si': {'name': 'si', 'lev': 6},
        'thetao': {'name': 'thetao', 'lev': 6},
        'mlotst': {'name': 'mlotst'},
        'dfe': {'name': 'dfe', 'lev': 6},
        'zmicroos': {'name': 'zmicroos'},
        'zmesoos': {'name': 'zmesoos'},
        'rsntds': {'name': 'rsntds'},
        'umo': {'name': 'umo', 'lev': 6},
        'vmo': {'name': 'vmo', 'lev': 6},
        'wmo': {'name': 'wmo', 'lev': 6},
    }
}

k_factors = {
    'intpp': 5E9,
    'no3': 5E3,
    'po4': 5E4,
    'si': 3E3,
    'thetao': 2E1,
    'mlotst': 1,
    'dfe': 3E7,
    'zmicroos': 2E4,
    'zmesoos': 1E5,
    'rsntds': 3E-2,
    'umo': 1E-8,
    'vmo': 1E-8,
    'wmo': 1E-8,
}


geo_locations = {
    'geo_points': {
        (120, 290): 'NORTH. POLAR PACIF.',
        (150, 150): 'TROP. PACIF.',
        (200, 150): 'GALAPAGOS',
        (250, 200): 'SUBTRP. NOR. ATLNT.',
        (100, 200): 'NORTH. PACIF.',
        (210, 250): 'LABRADOR',
        (350, 150): 'INDIAN',
        (50, 70): 'SO. AUSTRAL.',
        (150, 100): 'SO. PACIF.',
        (350, 100): 'SO. INDIA',
        (300, 70): 'SO. ATLANTIC',
    }}


In [ ]:
%%capture --no-stderr --no-display

start_time_extractor = time.time()

# Instantiate the class
warnings.filterwarnings('ignore')
extractor = TimeSeriesExtractor(datasets, k_factors,1959,1989,1990,1999,geo_locations)
warnings.filterwarnings('ignore' , category=UserWarning)
print("extractor ",extractor)
# List of index pairs




# Measure time taken to extract and normalize time series
normalized_time_series, execution_time = extractor.time_function_execution(extractor.get_time_series_for_multiple_points, index_pairs)

# Convert the time series data to DataFrames
df_train, df_test = extractor.convert_to_dataframe(normalized_time_series)

print(df_train.head())
print(df_test.head())
print(df_train.tail())
print(df_test.tail())

# After calculating cross-correlations
cross_corr_df = extractor.calculate_cross_correlations(normalized_time_series, max_lag=3)

print("cross_corr_df.head()",cross_corr_df.head())

# Save the DataFrame to a CSV file
cross_corr_df.to_csv('cross_correlation_results.csv', index=False)


end_time_extractor = time.time()
time_used_extractor = (end_time_extractor - start_time_extractor)
print("Time used for Extractor Class TimeSeriesExtractor(datasets, k_factors) ..../zzzz", f'{time_used_extractor:.4f}', "Seconds")
print("Time used for Extractor Class TimeSeriesExtractor(datasets, k_factors)..../zzzz", f'{time_used_extractor/60:.4f}', "Minutes")
print("Time used for Extractor Class TimeSeriesExtractor(datasets, k_factors)..../zzzz", f'{time_used_extractor/3600:.4f}', "Hours")

# Open a file in write mode
with open(f"{parent_dir}/output_time/Heatmap_{epoch_range}/output_time_heatmap_{epoch_range}_load.txt", "w") as file:
    # Redirect print output to the file
    print("Time used for LOAD..../zzzz", f'{time_used_extractor:.4f}', "Seconds", file=file)
    print("Time used for LOAD..../zzzz", f'{time_used_extractor/60:.4f}', "Minutes", file=file)
    print("Time used for LOAD..../zzzz", f'{time_used_extractor/3600:.4f}', "Hours", file=file)

df_train.head()


In [ ]:

%%capture --no-stderr --no-display
# Load the data
df = cross_corr_df

def plot_correlations(df):
    # Create a unique list of (point, var1, var2) combinations
    unique_combinations = df[['point', 'var1', 'var2']].drop_duplicates()

    # Iterate over each unique (point, var1, var2) combination and create a separate plot
    for index, row in unique_combinations.iterrows():
        point, var1, var2 = row['point'], row['var1'], row['var2']
        
        # Filter data for the specific combination
        subset = df[(df['point'] == point) & (df['var1'] == var1) & (df['var2'] == var2)]
        
        # Plot
        plt.figure(figsize=(10, 6))
        plt.plot(subset['lag'], subset['correlation'], marker='o', linestyle='-', label=f'{var1} vs {var2}', color=plt.cm.tab10(index % 10))
        plt.xlabel('Lag')
        plt.ylabel('Correlation')
        plt.title(f'Correlation between {var1} and {var2} at {point}')
        plt.legend()
        
        # Save the figure
        filename = f"{parent_dir}/plots/heatmap_correlation_plot_{point}_{var1}_vs_{var2}.png"
        plt.savefig(filename)
        plt.close()

# Run the function to generate and save the plots
plot_correlations(df)


In [ ]:
area_dict = index_pairs

for area in area_dict:
    extractor.add_lagged_series(df_train, df_test, 'intpp', area)
df_train.head()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
def apply_cross_correlation(series1, series2):
        series1 = np.array(series1)
        series2 = np.array(series2)
        

       

        corr = scipy.stats.pearsonr(series1,series2)
        cross_corr = corr.statistic
      
        
        return cross_corr



# LSTM
**Using the Long Short Term Memory for prediction** 
- Define the LSTM model architecture
- model = Sequential() For timeseries the Sequential Keras class is suitable
- model.add(LSTM(64, input_shape=(time_steps, 1))) Add LSTM cell into the Sequential architectur
- model.add(Dropout(0.2))  # Add dropout layer with a dropout rate of 0.2
- model.add(Dense(1)) Predict the next step of the time series
- model.compile(optimizer='adam', loss='mean_squared_error') At the end the model will be compiled with an Adam optimizer (For Gradient descent to reach the cost function) and the loss (cost function) of Mean Square Error (Similar to Least Square Fit).

# Train the model
 - model.fit(X_train, y_train, epochs=50, batch_size=16) As model is compiled, then a fit can be performed on the training data set with 50 epochs (runs for convergence) taking 16 data points as batches (at once) for running moving averags

In [ ]:



start_time_lstm = time.time()

"""intpp_raw_train_(120, 290)
intpp_normalized_train_(120, 290)
intpp_min_max_train_(120, 290)
intpp_log_train_(120, 290)
intpp_k_scaled_train_(120, 290)
no3_raw_train_(120, 290)
no3_normalized_train_(120, 290)
no3_min_max_train_(120, 290)
no3_log_train_(120, 290)
no3_k_scaled_train_(120, 290)	
si_normalized_train_(120, 290)
si_min_max_train_(120, 290)
si_log_train_(120, 290)
si_k_scaled_train_(120, 290)
intpp_lag1_train_(120, 290)
intpp_lag1_raw_train_(120, 290)
intpp_lag1_normalized_train_(120, 290)
intpp_lag1_min_max_train_(120, 290)
intpp_lag1_log_train_(120, 290)
intpp_lag1_k_scaled_train_(120, 290)"""

test_exp = f'{parent_dir}/plots/LSTM/MULTI_13210'
n_forecast_steps = 60

# Prepare input-output pairs for LSTM
def create_dataset(data, look_back):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        Y.append(data[i + look_back, 0])  # Assuming predicting Y
    return np.array(X), np.array(Y)

# Function to compute RMSE
def compute_rmse(y_true, y_pred):
    
    
    
    # Approach 1: Check if any NaN values exist in the DataFrame
    has_nan_pred = np.isnan(y_pred).all()
    has_nan_true = np.isnan(y_true).all()
    
    if has_nan_pred == True or has_nan_true == True:

        print(f"DataFrame contains NaN values y_pred: {has_nan_pred}")

        # Approach 2: Check for NaN values in each column and the entire DataFrame
        nan_in_columns = y_pred.isnull().any()
        print("NaN values in each column y_pred:")
        print(nan_in_columns)

        nan_in_any = y_pred.isnull().any().any()
        print(f"DataFrame contains NaN valuesy_pred : {nan_in_any}")

        # Approach 3: Count NaN values in each column and the total count
        nan_count = y_pred.isnull().sum()
        print("Count of NaN values in each column y_pred:")
        print(nan_count)

        total_nan_count = y_pred.isnull().sum().sum()
        print(f"Total count of NaN values in DataFrame y_pred: {total_nan_count}")

        print(f"DataFrame contains NaN values y_true: {has_nan_true}")

        # Approach 2: Check for NaN values in each column and the entire DataFrame
        nan_in_columns = y_true.isnull().any()
        print("NaN values in each column y_true:")
        print(nan_in_columns)

        nan_in_any = y_true.isnull().any().any()
        print(f"DataFrame contains NaN values y_true : {nan_in_any}")

        # Approach 3: Count NaN values in each column and the total count
        nan_count = y_true.isnull().sum()
        print("Count of NaN values in each column y_true:")
        print(nan_count)

        total_nan_count = y_true.isnull().sum().sum()
        print(f"Total count of NaN values in DataFrame y_true: {total_nan_count}")
        

    if has_nan_pred == False:
        if has_nan_true == False:
    
            mae = mean_absolute_error(y_true, y_pred)
            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_true, y_pred)
            print("="*100)
            print(" RMSE FUNCTION ")
            print("y_true",y_true)
            print("y_pred_transposed",y_pred.transpose())
            print("mean y_true", np.mean(y_true))
            print("mean y_pred", np.mean(y_pred))
            print("MSE :", mse)
            print("MAE :", mae)
            print("RMSE :", rmse)
            print("R2 score :", r2)


            rmse_percent = rmse/np.abs(np.mean(y_true))*100
            
            print(f"rmse vs y_true {rmse: .2e} vs. {np.mean(y_true): .2e}")
            
            print("MSE %:", mse/np.mean(y_true)*100)
            print("MAE %:", mae/np.mean(y_true)*100)
            print("RMSE %:", rmse_percent)
            print("="*100)
    else:
        
        rmse_percent = np.nan
    
    
    
    return rmse_percent




normx_dict = ['normalized','k_scaled']


# Columns to add stepwise
columns = ['A' ,'B','C','D', 'E', 'F', 'G', 'H', 'I','J','K','L']

# DataFrame to store RMSE values
rmse_results = pd.DataFrame(columns=['NORM', 'COLUMNS', 'VAR','RMSE','AREA','GEO','MODEL','HYPERTUNE','CORR','CORR_TRANSF'])




# Mapping for train and test columns

variables = {
    'A': 'no3',
    'B': 'po4',
    'C': 'si',
    'D': 'thetao',
    'E': 'mlotst',
    'F': 'dfe',
    'G': 'zmicroos',
    'H': 'zmesoos',
    'I': 'rsntds',
    'J': 'umo',
    'K': 'vmo',
    'L': 'wmo'
}






for area in area_dict:
    
    
    
    print('RUN FOR FOLLOWING LOCATION ', area)
    area = str(area)


    for normx in normx_dict:
        
        
        
        
        
        train_columns = {
            'A': 'no3_'+normx+'_train_'+area,
            'B': 'po4_'+normx+'_train_'+area,
            'C': 'si_'+normx+'_train_'+area,
            'D': 'thetao_'+normx+'_train_'+area,
            'E': 'mlotst_'+normx+'_train_'+area,
            'F': 'dfe_'+normx+'_train_'+area,
            'G': 'zmicroos_'+normx+'_train_'+area,
            'H': 'zmesoos_'+normx+'_train_'+area,
            'I': 'rsntds_'+normx+'_train_'+area,
            'J': 'umo_'+normx+'_train_'+area,
            'K': 'vmo_'+normx+'_train_'+area,
            'L': 'wmo_'+normx+'_train_'+area
        }

        test_columns = {
            'A': 'no3_'+normx+'_test_'+area,
            'B': 'po4_'+normx+'_test_'+area,
            'C': 'si_'+normx+'_test_'+area,
            'D': 'thetao_'+normx+'_test_'+area,
            'E': 'mlotst_'+normx+'_test_'+area,
            'F': 'dfe_'+normx+'_test_'+area,
            'G': 'zmicroos_'+normx+'_test_'+area,
            'H': 'zmesoos_'+normx+'_test_'+area,
            'I': 'rsntds_'+normx+'_test_'+area,
            'J': 'umo_'+normx+'_test_'+area,
            'K': 'vmo_'+normx+'_test_'+area,
            'L': 'wmo_'+normx+'_test_'+area
        }


        #normx = normx_dict[i]

        df_test_predictor = df_test['intpp_'+normx+'_test_'+area][0:]
        df_train_predictor_shift = df_train['intpp_'+normx+'_train_'+area][1:]
        df_train_predictor = df_train['intpp_'+normx+'_train_'+area][0:]

        # Create new DataFrames for dependent and independent variables for multivariate of Z, A, B

        df_new = pd.DataFrame(df_train_predictor, columns=['Z'])

        df_new['Z'] = df_train_predictor_shift
        df_new['Zt-1'] = df_train['intpp_lag1_'+normx+'_train_'+area][1:]

        #print(df_new.head())


        # Create new DataFrames for dependent and independent variables for multivariate of Z, A, B
        df_new_test = pd.DataFrame(df_test_predictor, columns=['Z'])
        df_new_test['Z'] = df_test_predictor
        df_new_test['Zt-1'] = df_test['intpp_lag1_'+normx+'_test_'+area]


        # Histogram
        plt.figure(figsize=(10, 6))
        plt.hist(df_new['Z'])
        plt.hist(df_new['Zt-1'])
        plt.title(f'ML LSTM Multvar. INTPP  {normx.upper()} @ {area}')
        plt.xlabel('Data value')
        plt.ylabel('#N')
        plt.legend('INTPP')
        
        print("Root Path for Experiment: ",test_exp)
        print("os.path.exists(test_exp)???",os.path.exists(test_exp))
        if os.path.exists(test_exp) == False:
            print("!mkdir 'plots/LSTM/MULTI_'+test_exp")
            os.makedirs(test_exp, exist_ok=True)
            print("os.path.exists(test_exp)!!!",os.path.exists(test_exp))
        
        if os.path.exists('plots/LSTM/MULTI_'+test_exp+'/HIST') == False:
            os.makedirs(test_exp+'/HIST', exist_ok=True) 
        plt.savefig(test_exp+f'/HIST/HISTOGRAM_var_0_INTPP_{normx}_{area}.png') 




        


        # Iterate through columns and expand the DataFrame stepwise
        for col in columns:
            n_var = str(columns.index(col)+1)
            df_new[col] = df_train[train_columns[col]][1:]
            df_new_test[col] = df_test[test_columns[col]]
            
            df_new.fillna(0)
            df_new_test.fillna(0)
            # Evaluate the DataFrame at each step (you can add your own evaluation code here)
            print(f"Evaluating with columns: ['Z', 'Zt-1'] + {columns[:columns.index(col)+1]}")
            #print(df_new.head())
            #print(df_new_test.head())

        
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(df_new)
            scaled_test_data = scaler.fit_transform(df_new_test)



            print(np.shape(scaled_data))
            print(np.shape(scaled_test_data))

            # Validation and forecasting steps omitted for brevity



            look_back = 12  # Number of time steps to look back
            X, Y = create_dataset(scaled_data, look_back)

            # Define and train the LSTM model
            model = Sequential()
            model.add(LSTM(units=50, input_shape=(X.shape[1], X.shape[2])))

            # Add dense layers
            model.add(Dense(units=32, activation='relu'))
            model.add(Dense(units=1, activation='linear'))  # Assuming you are predicting a single output
            model.add(Dropout(0.1))
            model.add(Dense(units=1))
            model.compile(optimizer='adam', loss='mean_squared_error')

            

            
            

            # Train the model and store the training history
            from keras.callbacks import EarlyStopping
            # Define the EarlyStopping callback
            #early_stopping = EarlyStopping(monitor='loss', patience=8, verbose=1)

            # Fit the model with EarlyStopping callback
            history = model.fit(X, Y, epochs=100, batch_size=32, verbose=0)#, validation_split=0.2)
            #history = model.fit(X, Y, epochs=100, batch_size=32,callbacks=[early_stopping])

            # Access the loss values from the history object
            loss = history.history['loss']
            #acc = history.history['accuracy']
            # Plot the loss curve
            
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(loss)), loss)
            plt.xlabel('Epochs')
            plt.ylabel('Loss: Mean Squared Error')
            plt.title(f'Loss_{n_var}_INTPP_to_{variables[col]}_{normx}_{area}')
            if os.path.exists(test_exp+'/LOSS') == False:
                os.makedirs(test_exp+'/LOSS', exist_ok=True) 
            plt.savefig(test_exp+f'/LOSS/LOSS_{n_var}_INTPP_to_{variables[col]}_{normx}_{area}.png') 
           



            # Transform the values into the scaled 0..1
            scaled_test_data = scaler.fit_transform(df_new_test)


            X_test, Y_test = create_dataset(scaled_test_data, look_back)




            #How many steps do you want predict?

            X_test = X_test[0:n_forecast_steps-12]
            Y_test = Y_test[0:n_forecast_steps-12]



            # Given minimum and maximum values per feature for the first column
            min_val = scaler.data_min_[0]
            max_val = scaler.data_max_[0]


            # Generate predictions on the scaled test data
            y_pred = model.predict(X_test)





            y_pred_transformed = y_pred*(max_val - min_val)+min_val
            Y_test_transformed = Y_test*(max_val - min_val)+min_val
             # Calculate RMSE
            rmse = compute_rmse(Y_test_transformed, y_pred_transformed)
            print("RMSE Transfomred FLAT", rmse)
            # Histogram
            plt.figure(figsize=(10, 6))
            plt.hist(df_new[col])
            plt.title(f' Histogram {variables[col].upper()} & {normx.upper()} @ {area} forecast. RMSE:= {rmse: .2f} %')
            plt.xlabel('Data value')
            plt.ylabel('#N')
            print("variables[col].upper()",variables[col].upper())
            plt.legend(variables[col].upper())
            if os.path.exists(test_exp+'/HIST') == False:
                os.makedirs(test_exp+'/HIST', exist_ok=True) 
            plt.savefig(test_exp+f'/HIST/HISTOGRAM_var_{variables[col]}_{normx}_{area}.png') 
            

            
           
            
            y_pred_flat = y_pred.flatten()
            
       
            
            y_pred_transformed_flat = y_pred_transformed.flatten()
            
            
            
            corr_forecast_orig = apply_cross_correlation(y_pred_flat,Y_test)
          
            corr_forecast_transformed = apply_cross_correlation(y_pred_transformed_flat,Y_test_transformed)
    
            
            
           
            import ast
            
        
            
            
            # Calculate RMSE
            rmse = compute_rmse(Y_test_transformed, y_pred_transformed_flat)
            print("RMSE Transfomred FLAT", rmse)
  
            
            # Store the RMSE value in the DataFrame
            new_row = pd.DataFrame({'NORM': [normx], 'COLUMNS': [columns[:columns.index(col)+1]],
                                    'VAR': [variables[col]],
                                    'RMSE': [rmse],'AREA': [area],'GEO': geo_locations['geo_points'][ast.literal_eval(area)],'MODEL': 'LSTM','HYPERTUNE':'50,32,0.1,1,1',
                                   'CORR': [corr_forecast_orig],'CORR_TRANSF': [corr_forecast_transformed]}
                                  )
            rmse_results = pd.concat([rmse_results, new_row], ignore_index=True)


            
            
            
            
            plt.figure(figsize=(10, 6))
            # Plot the actual values and the predictions
            plt.plot(Y_test_transformed, label='Test sequence',linestyle='dotted')
            plt.plot(y_pred_transformed, label='Predicted sequence')
            plt.title(f'ML LSTM Multvar. INTPP to {variables[col].upper()} &  {normx.upper()} @ {area} forecast. RMSE:= {rmse: .2f} %')
            plt.xlabel('Time')
            plt.ylabel('Re-Scaled values [ ]')
            plt.legend()
            if os.path.exists(test_exp+'/FORECAST') == False:
                os.makedirs(test_exp+'/FORECAST', exist_ok=True) 
            
            plt.savefig(test_exp+f'/FORECAST/FORECAST_var_{n_var}_INTPP_to_{variables[col]}_{normx}_{area}.png') 

            #plt.show()

# Save the RMSE results to a CSV file

stamp = datetime.now().strftime("%Y-%m-%d_%HH:%MM:%SS")

rmse_results.to_csv(test_exp+f'/LSTM_RMSE_{stamp}.csv', index=False)  



df = rmse_results

# Separate heatmaps for 'normalized' and 'k_scaled'
for norm_type in df['NORM'].unique():
    # Filter the DataFrame by 'NORM' value
    df_filtered = df[df['NORM'] == norm_type]
    
    # Pivot the table to get 'AREA' on the y-axis, 'VAR' on the x-axis, and 'CORR' as values
    heatmap_data = df_filtered.pivot(index='AREA', columns='VAR', values='CORR')
    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data, annot=True, cmap="coolwarm",
                vmin=-1,vmax=1,
                cbar_kws={'label': 'Correlation'})
    plt.title(f'Corr. Heatmap of Area vs Variables for {norm_type} from LSTM model')
    plt.xlabel('Variables')
    plt.ylabel('Area')
    
    # Save the heatmap
    plt.savefig(test_exp+f'/Heatmap_LSTM_CORR_{norm_type}_INDEX.png')
    
    
    
    
    
    heatmap_data = df_filtered.pivot(index='GEO', columns='VAR', values='CORR')
    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data, annot=True, cmap="coolwarm",
                vmin=-1, vmax=1, cbar_kws={'label': 'Correlation'})
    plt.title(f'Corr. Heatmap of Geolocation vs Variables for {norm_type} from LSTM model')
    plt.xlabel('Variables')
    plt.ylabel('Geolocation')
    
    # Save the heatmap
    plt.savefig(test_exp+f'/Heatmap_LSTM_CORR_{norm_type}_GEO.png')


    # Pivot the table to get 'AREA' on the y-axis, 'VAR' on the x-axis, and 'CORR' as values
    heatmap_data = df_filtered.pivot(index='AREA', columns='VAR', values='RMSE')
    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data, annot=True, cmap="coolwarm",
                vmin=0,vmax=20,cbar_kws={'label': 'RMSE %'})
    plt.title(f'RMSE Heatmap of Area vs Variables for {norm_type} from LSTM model')
    plt.xlabel('Variables')
    plt.ylabel('Area')
    
    # Save the heatmap
    plt.savefig(test_exp+f'/Heatmap_LSTM_RMSE_{norm_type}_INDEX.png')
    
    # Pivot the table to get 'AREA' on the y-axis, 'VAR' on the x-axis, and 'CORR' as values
    heatmap_data = df_filtered.pivot(index='GEO', columns='VAR', values='RMSE')
    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data, annot=True, cmap="coolwarm",
                vmin=0,vmax=20,cbar_kws={'label': 'RMSE %'})
    plt.title(f'RMSE Heatmap of Area vs Variables for {norm_type} from LSTM model')
    plt.xlabel('Variables')
    plt.ylabel('Area')
    
    # Save the heatmap
    plt.savefig(test_exp+f'/Heatmap_LSTM_RMSE_{norm_type}_GEO.png')
    
    
    

end_time_lstm = time.time()

time_used_lstm = end_time_lstm - start_time_lstm

# Print the model summary
model.summary()

print("Time used for LSTM ..../zzzz", f'{time_used_lstm:.4f}', "Seconds")
print("Time used for LSTM..../zzzz", f'{time_used_lstm/60:.4f}', "Minutes")
print("Time used for LSTM..../zzzz", f'{time_used_lstm/3600:.4f}', "Hours")


In [ ]:
print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook:.4f}', "Seconds")
print("Time used for entire LOAD..../zzzz", f'{time_used_load:.4f}', "Seconds")
#print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot:.4f}', "Seconds")
#print("Time used for entire PLOTSERIES", f'{time_used_plot1:.4f}', "Seconds")
print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor:.4f}', "Seconds")

print("Time used for entire LSTM.../zzzz", f'{time_used_lstm:.4f}', "Seconds")





print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/60:.4f}', "Minutes")
print("Time used for entire LOAD..../zzzz", f'{time_used_load/60:.4f}', "Minutes")
#print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/60:.4f}', "Minutes")
#print("Time used for entire PLOTSERIES", f'{time_used_plot1/60:.4f}', "Minutes")
print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/60:.4f}', "Minutes")

print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/60:.4f}', "Minutes")

print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/3600:.4f}', "Hours")
print("Time used for entire LOAD..../zzzz", f'{time_used_load/3600:.4f}', "Hours")
#print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/3600:.4f}', "Hours")
#print("Time used for entire PLOTSERIES", f'{time_used_plot1/3600:.4f}', "Hours")
print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/3600:.4f}', "Hours")

print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/3600:.4f}', "Hours")

print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/time_used_notebook*100:.2f}', "%")
print("Time used for entire LOAD..../zzzz", f'{time_used_load/time_used_notebook*100:.2f}', "%")
#print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/time_used_notebook*100:.2f}', "%")
#print("Time used for entire PLOT SERIES..../zzzz", f'{time_used_plot1/time_used_notebook*100:.2f}', "%")
print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/time_used_notebook*100:.2f}', "%")

print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/time_used_notebook*100:.2f}', "%")


# Open a file in write mode
with open(f"{parent_dir}/output_time/Heatmap_{epoch_range}/output_time_heatmap_{epoch_range}_overview_1.txt", "w") as file:
    # Redirect print output to the file
    print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook:.4f}', "Seconds", file=file)
    print("Time used for entire LOAD..../zzzz", f'{time_used_load:.4f}', "Seconds", file=file)
    #print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot:.4f}', "Seconds")
    #print("Time used for entire PLOTSERIES", f'{time_used_plot1:.4f}', "Seconds")
    print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor:.4f}', "Seconds", file=file)

    print("Time used for entire LSTM.../zzzz", f'{time_used_lstm:.4f}', "Seconds", file=file)





    print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/60:.4f}', "Minutes", file=file)
    print("Time used for entire LOAD..../zzzz", f'{time_used_load/60:.4f}', "Minutes", file=file)
    #print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/60:.4f}', "Minutes")
    #print("Time used for entire PLOTSERIES", f'{time_used_plot1/60:.4f}', "Minutes")
    print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/60:.4f}', "Minutes", file=file)

    print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/60:.4f}', "Minutes", file=file)

    print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/3600:.4f}', "Hours", file=file)
    print("Time used for entire LOAD..../zzzz", f'{time_used_load/3600:.4f}', "Hours", file=file)
    #print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/3600:.4f}', "Hours")
    #print("Time used for entire PLOTSERIES", f'{time_used_plot1/3600:.4f}', "Hours")
    print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/3600:.4f}', "Hours", file=file)

    print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/3600:.4f}', "Hours", file=file)

    print("Time used for entire NOTEBOOK..../zzzz", f'{time_used_notebook/time_used_notebook*100:.2f}', "%", file=file)
    print("Time used for entire LOAD..../zzzz", f'{time_used_load/time_used_notebook*100:.2f}', "%", file=file)
    #print("Time used for entire PLOT2D..../zzzz", f'{time_used_plot/time_used_notebook*100:.2f}', "%", file=file)
    #print("Time used for entire PLOT SERIES..../zzzz", f'{time_used_plot1/time_used_notebook*100:.2f}', "%")
    print("Time used for entire EXTRACTOR.../zzzz", f'{time_used_extractor/time_used_notebook*100:.2f}', "%", file=file)

    print("Time used for entire LSTM.../zzzz", f'{time_used_lstm/time_used_notebook*100:.2f}', "%", file=file)

